In [1]:
import pandas as pd
import csv
import numpy as np

In [2]:
train = 'train.txt'
labels = 'train_labels.txt'

In [28]:
train_df = pd.read_csv(train, sep=" ", engine="c", header=None)

In [29]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9
0,120.3882,67.1368,3.6905,0.1128,0.0481,-155.5746,35.3004,47.8095,3.8760,289.0430
1,28.5322,15.1782,2.6686,0.4306,0.2399,30.9627,21.7936,21.9248,62.7504,254.0526
2,38.8799,21.5881,3.0573,0.2025,0.1047,18.7566,-24.0106,11.6233,0.7040,185.2510
3,28.8255,16.7964,2.6642,0.3402,0.1701,3.6149,-18.2080,9.8634,2.4949,213.1690
4,20.7522,8.9492,2.3874,0.6434,0.3586,24.0649,10.5927,1.8768,36.3060,126.3930


In [5]:
train_df.shape

(15000, 10)

In [6]:
labels_df = pd.Series(pd.read_csv(labels, sep=" ", engine="c", header=None)[0])

In [7]:
labels_df.head()

0    1
1    1
2    0
3    0
4    1
Name: 0, dtype: int64

Normalization:

In [8]:
from sklearn import preprocessing
train_df = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(train_df))

In [9]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9
0,0.346150,0.261862,0.516422,0.113107,0.070825,0.287144,0.643887,0.673694,0.043067,0.582183
1,0.065218,0.059202,0.213772,0.474366,0.355016,0.469097,0.620195,0.604959,0.697227,0.511392
2,0.096865,0.084203,0.328891,0.215073,0.154690,0.457191,0.539851,0.577604,0.007822,0.372196
3,0.066115,0.065513,0.212469,0.371604,0.251593,0.442421,0.550029,0.572931,0.027721,0.428678
4,0.041424,0.034906,0.130490,0.716267,0.530893,0.462369,0.600547,0.551723,0.403400,0.253117


### KNN

In [12]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

Results *before* parameter search:

In [13]:
knn_grid_naive = GridSearchCV(KNeighborsClassifier(), param_grid={})
knn_grid_naive.fit(train_df, labels_df)
print(knn_grid_naive.best_score_)
print(knn_grid_naive.best_params_)

0.8306
{}


In [14]:
print('Mean cross-validation error:', 1.0 - knn_grid_naive.best_score_)

Mean cross-validation error: 0.1694


Results *after* parameter search:

In [32]:
from sklearn.neighbors import KNeighborsClassifier

knn_neighbors = [
#     7,
    16
]
for i in range(10,50,5):
    knn_neighbors.append(i+1)

knn_cv_params = {
    'n_neighbors': knn_neighbors,
#     'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2, 3],
    'weights': ['uniform', 'distance']
}

knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid=knn_cv_params)
knn_grid.fit(train_df, labels_df)
print(knn_grid.best_score_)
print(knn_grid.best_params_)

0.8411333333333333
{'p': 1, 'n_neighbors': 16, 'weights': 'distance'}


In [33]:
print('Mean cross-validation error:', 1.0 - knn_grid.best_score_)

Mean cross-validation error: 0.1588666666666667


### SVM

Results *before* parameter search:

In [21]:
svm_grid_naive = GridSearchCV(SVC(), param_grid={})
svm_grid_naive.fit(train_df, labels_df)
print(svm_grid_naive.best_score_)
print(svm_grid_naive.best_params_)

0.8166666666666667
{}


In [34]:
print('Mean cross-validation error:', 1.0 - svm_grid_naive.best_score_)

Mean cross-validation error: 0.18333333333333335


Results *after* parameter search:

In [12]:
from sklearn.svm import SVC

svm_cv_params = {
#     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'probability': [True, False]
#     'C': [0.1, 0.5, 1.0]
}

svm_grid = GridSearchCV(SVC(cache_size=512, C=15.0, kernel='rbf'), param_grid=svm_cv_params)
svm_grid.fit(train_df, labels_df)
print(svm_grid.best_score_)
print(svm_grid.best_params_)

0.8408
{'probability': True}


In [35]:
print('Mean cross-validation error:', 1.0 - svm_grid.best_score_)

Mean cross-validation error: 0.1592


### Random Forest

Results *before* parameter search:

In [22]:
rf_grid_naive = GridSearchCV(RandomForestClassifier(), param_grid={})
rf_grid_naive.fit(train_df, labels_df)
print(rf_grid_naive.best_score_)
print(rf_grid_naive.best_params_)

0.8656666666666667
{}


In [36]:
print('Mean cross-validation error:', 1.0 - rf_grid_naive.best_score_)

Mean cross-validation error: 0.1343333333333333


Results *after* parameter search:

In [13]:
from sklearn.ensemble import RandomForestClassifier

rf_cv_params = {
    'n_estimators': [100, 300, 500],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None]
}

rf_grid = GridSearchCV(RandomForestClassifier(), param_grid=rf_cv_params)
rf_grid.fit(train_df, labels_df)
print(rf_grid.best_score_)
print(rf_grid.best_params_)

0.8763333333333333
{'n_estimators': 100, 'criterion': 'gini', 'max_features': 'sqrt'}


In [37]:
print('Mean cross-validation error:', 1.0 - rf_grid.best_score_)

Mean cross-validation error: 0.1236666666666667


### Gradient Boosting Trees

Results *before* parameter search:

In [23]:
gbt_grid_naive = GridSearchCV(GradientBoostingClassifier(), param_grid={})
gbt_grid_naive.fit(train_df, labels_df)
print(gbt_grid_naive.best_score_)
print(gbt_grid_naive.best_params_)

0.8700666666666667
{}


In [38]:
print('Mean cross-validation error:', 1.0 - gbt_grid_naive.best_score_)

Mean cross-validation error: 0.12993333333333335


Results *after* parameter search:

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

gbt_cv_params = {
    'loss': ['deviance', 'exponential'],
#     'n_estimators': [100, 300, 500, 700],
    'n_estimators': [700, 900, 1100],
    'max_features': ['sqrt', 'log2', None]
}

gbt_grid = GridSearchCV(GradientBoostingClassifier(), param_grid=gbt_cv_params)
gbt_grid.fit(train_df, labels_df)
print(gbt_grid.best_score_)
print(gbt_grid.best_params_)

0.8823333333333333
{'n_estimators': 1100, 'max_features': 'log2', 'loss': 'deviance'}


In [39]:
print('Mean cross-validation error:', 1.0 - gbt_grid.best_score_)

Mean cross-validation error: 0.1176666666666667


### Best classifier

From the cross-validation, the best found classifier is `GradientBoostingClassifier()`

With params:

In [24]:
gbt_grid.best_params_

{'loss': 'deviance', 'max_features': 'log2', 'n_estimators': 1100}

Create instance of classifier with best found parameters and predict classes for test data:

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
best_clf = GradientBoostingClassifier(n_estimators=1100, loss='deviance', max_features='log2')

In [19]:
shape_of_train = train_df.shape
shape_of_train

(15000, 10)

In [31]:
test = 'test.txt'
test_df = pd.read_csv(test, sep=" ", engine="c", header=None)
# test_df = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(test_df))
test_df.head()

,0,1,2,3,4,5,6,7,8,9
0,16.1965,11.5351,2.2878,0.6082,0.3686,1.7609,-5.1272,-7.0963,10.7960,134.3410
1,50.9418,21.5709,3.2002,0.2315,0.1328,-16.3625,-44.7336,2.4560,2.8444,229.1480
2,10.6118,10.0548,2.2398,0.5855,0.3150,-2.8825,0.5839,4.4629,44.4008,187.5178
3,105.2990,63.2194,3.9254,0.1333,0.0709,26.6490,56.1306,-31.8837,25.8700,366.2470
4,36.2652,16.2752,2.6803,0.2651,0.1347,-1.7525,-15.3032,10.9965,5.0774,134.8040


Train/Test normalization to have correct generalized parameters:

In [32]:
train_test_df = pd.concat([train_df, test_df])
train_test_df.head()

,0,1,2,3,4,5,6,7,8,9
0,120.3882,67.1368,3.6905,0.1128,0.0481,-155.5746,35.3004,47.8095,3.8760,289.0430
1,28.5322,15.1782,2.6686,0.4306,0.2399,30.9627,21.7936,21.9248,62.7504,254.0526
2,38.8799,21.5881,3.0573,0.2025,0.1047,18.7566,-24.0106,11.6233,0.7040,185.2510
3,28.8255,16.7964,2.6642,0.3402,0.1701,3.6149,-18.2080,9.8634,2.4949,213.1690
4,20.7522,8.9492,2.3874,0.6434,0.3586,24.0649,10.5927,1.8768,36.3060,126.3930


In [33]:
from sklearn import preprocessing
train_test_df = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(train_test_df))
train_test_df.head()

,0,1,2,3,4,5,6,7,8,9
0,0.351946,0.261862,0.517209,0.113308,0.070825,0.292639,0.643887,0.657698,0.043067,0.582183
1,0.073505,0.059202,0.215050,0.474486,0.355016,0.473189,0.620195,0.590595,0.697227,0.511392
2,0.104871,0.084203,0.329982,0.215252,0.154690,0.461375,0.539851,0.563890,0.007822,0.372196
3,0.074394,0.065513,0.213749,0.371747,0.251593,0.446719,0.550029,0.559327,0.027721,0.428678
4,0.049921,0.034906,0.131904,0.716331,0.530893,0.466513,0.600547,0.538623,0.403400,0.253117


In [26]:
print(train_df.shape)
print(test_df.shape)
print(train_test_df.shape)

(15000, 10)
(4020, 10)
(19020, 10)


In [35]:
train_df = train_test_df[:shape_of_train[0]]
test_df = train_test_df[shape_of_train[0]:]

In [37]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9
0,0.351946,0.261862,0.517209,0.113308,0.070825,0.292639,0.643887,0.657698,0.043067,0.582183
1,0.073505,0.059202,0.215050,0.474486,0.355016,0.473189,0.620195,0.590595,0.697227,0.511392
2,0.104871,0.084203,0.329982,0.215252,0.154690,0.461375,0.539851,0.563890,0.007822,0.372196
3,0.074394,0.065513,0.213749,0.371747,0.251593,0.446719,0.550029,0.559327,0.027721,0.428678
4,0.049921,0.034906,0.131904,0.716331,0.530893,0.466513,0.600547,0.538623,0.403400,0.253117


In [36]:
print(train_df.shape)
print(test_df.shape)

(15000, 10)
(4020, 10)


In [15]:
best_clf.fit(train_df, labels_df)

NameError: name 'best_clf' is not defined

Importance of each feature:

In [45]:
best_clf.feature_importances_

array([0.13541657, 0.12496926, 0.10984525, 0.09866443, 0.09646131,
       0.07260438, 0.10724194, 0.06020644, 0.09288759, 0.10170283])

In [51]:
test_labels_df = pd.Series(best_clf.predict(test_df))

In [52]:
test_labels_df.head()

0    1
1    0
2    1
3    1
4    0
dtype: int64

In [53]:
test_labels_df.to_csv('test_labels.csv', sep=',')